<a href="https://colab.research.google.com/github/tap-o-chek/Multiple-processing-course/blob/main/labs/MP_Lesson_1_%D0%96%D0%B8%D0%B4%D0%B5%D0%BB%D0%B5%D0%B2%D0%B0_%D0%A1%D0%BE%D1%84%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Упражнение 1.**

Перед вами фрагмент кода, содержащего некоторую проблему. Всегда ли counter = 10 после запуска программы?

In [ ]:
import threading
import sys

def thread_job():
    global counter
    old_counter = counter
    counter = old_counter + 1
    print('{} '.format(counter), end='')
    sys.stdout.flush()


counter = 0
threads = [threading.Thread(target=thread_job) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
print(counter)

1 2 3 4 5 6 7 8 9 10 10


Для наглядности продемонстрируем "проблему"

In [ ]:
import threading
import random
import time
import sys

def thread_job():
    global counter
    old_counter = counter
    time.sleep(random.randint(0, 1)) #задержка рандомной длительности
    counter = old_counter + 1
    print('{} '.format(counter), end='')
    sys.stdout.flush()


counter = 0
threads = [threading.Thread(target=thread_job) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
print(counter)

1 2 3 4 5 5 2 2 3 4 4


**Объясните почему так происходит?**

Этот код создает 10 потоков, каждый из которых выполняет одну функцию, которая инкрементирует глобальную переменную counter. Так как инкрементация counter происходит в нескольких потоках параллельно, возникает состязание за ресурс (гонка данных), что приводит к тому, что результат выполнения программы может быть непредсказуемым. В зависимости от того, какие потоки завершат свое выполнение первыми, и какие данные они прочитают в момент выполнения инкрементации, результат может быть разным, поэтому в решении мы накладываем локи, которые открывают доступ лишь одному потоку.

**Исправьте проблему.**


In [ ]:
import threading
import random
import time
import sys

def thread_job():
    global counter
    lock.acquire() # включаю блок
    old_counter = counter
    time.sleep(random.randint(0, 1)) # случайная задержка
    counter = old_counter + 1
    print('{} '.format(counter), end='')
    lock.release() # снимаю блок
    sys.stdout.flush()


counter = 0
lock = threading.Lock() # объект блокировки
threads = [threading.Thread(target=thread_job) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
print(counter)

1 2 3 4 5 6 7 8 9 10 10


**Упражнение 2.**

Программист хочет узнать доступность набора ip адресов. Он реализовал программу. Почему она неэффективна? Переделайте с использованием threading. Измерить время с применением потоков и без них. Объяснить результат.

In [ ]:
import os, re

received_packages = re.compile(r"(\d) received")

def status(x):
    if x == 0:
        return "no response"
    elif x == 1:
        return "losses"
    elif x == 2:
        return "alive"

time0 = time.time()

for suffix in range(0, 100):
    ip = "192.168.178." + str(suffix)
    ping_out = os.popen("ping -q -c2 " + ip, "r")  # получение вердикта
    #print("... pinging ", ip)
    while True:
        line = ping_out.readline()
        if not line:
            break
        n_received = received_packages.findall(line)
        #if n_received:
    print("Status: ", ip, status(-1))

print(f"time: ", time.time() - time0)

**Объясните:**

Проблема с текущей реализацией программы в том, что она выполняет пинг для каждого IP-адреса последовательно, что сильно замедляет процесс, особенно если множество адресов недоступно или имеет большую задержку. Чтобы сделать программу более эффективной, используем многопоточность для параллельного выполнения нескольких пингов.

А ещё, сама пинговка не работает и файл не читается, поэтому прогнав программу, не получится узнать, сработало или нет. Но вроде должно работать -.-

In [ ]:
import os, re
import threading
import time

received_packages = re.compile(r"(\d) received")

def status(x):
    if x == 0:
        return "no response"
    elif x == 1:
        return "losses"
    elif x == 2:
        return "alive"

time0 = time.time()
#threads = [threading.Thread(target=status, args=(suffix,)) for _ in range(100)]
threads = []  # создаем список для хранения потоков

for suffix in range(0, 100):
    ip = "192.168.178." + str(suffix)
    ping_out = os.popen("ping -q -c2 " + ip, "r")  # получение вердикта
    #print("... pinging ", ip)
    while True:
        line = ping_out.readline()
        if not line:
            break
        n_received = received_packages.findall(line)  # смотрим, сколько пакетов получено
        for _ in n_received:
          thread = threading.Thread(target=status, args=(line,))  # для каждого пакета запускаем поток на вычисление статуса
          thread.start()
          threads.append(thread)  # добавляем поток в список
        for thread in threads:
          thread.join()  # Ожидаем завершения всех потоков
    print("Status: ", ip, thread)

print(f"time: ", time.time() - time0)

**Упражнение 3.**

Составить программу, которая считает сумму элементов массива (создать из K значений и заполнить случайным образом) с использованием N потоков. Запустить с разным параметром N (2, 4, 8, 16, 32, 64). Объяснить результат (потребуется измерить время).

In [ ]:
import threading
import random
import time

def partial_sum(arr, start, end, result):
    partial_sum = sum(arr[start:end])  # Вычисляем сумму элементов в части массива
    result.append(partial_sum)  # Добавляем частичную сумму к общей сумме

def sum_with_threads(N, K):

    result = []  # хранение общей суммы
    threads = [] # для N потоков

    for i in range(N):
        start = i * (K // N)  # начальный индекс для каждого потока
        end = start + (K // N) if i != N - 1 else K  # конечный индекс для каждого потока (последний поток обрабатывает оставшуюся часть)
        thread = threading.Thread(target=partial_sum, args=(arr, start, end, result))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

    return sum(result) # суммируем элементы массива результата

K = 100000  # размер массива
arr = [random.randint(1, 100000) for _ in range(K)]  # массив из K случайных чисел

for N in [2, 4, 8, 16, 32, 64]:
    start_time = time.time()
    total_sum = sum_with_threads(N, K)  # вычисляем сумму с использованием N потоков
    end_time = time.time()
    print(f"Сумма с {N} потоками: {total_sum}")
    print(f"Время: {end_time - start_time}")
    print()

Сумма с 2 потоками: 5007894889
Время: 0.0026082992553710938

Сумма с 4 потоками: 5007894889
Время: 0.003667116165161133

Сумма с 8 потоками: 5007894889
Время: 0.0038433074951171875

Сумма с 16 потоками: 5007894889
Время: 0.0073888301849365234

Сумма с 32 потоками: 5007894889
Время: 0.008788824081420898

Сумма с 64 потоками: 5007894889
Время: 0.014769554138183594



Каждый поток считает свою сумму для своего N паралелльно, поэтому выходит быстрее.

**Упражнение 4.**

Запустите на исполнение следующий фрагмент кода, замерив время работы. Перепишите с помощью потоков и опять замерьте время. Объясните результат.

In [ ]:
import urllib.request
import time


urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


start = time.time()
for url in urls:
    read_url(url)
print(time.time() - start)

5.62566065788269


In [ ]:
import urllib.request
import time
import threading

urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]

def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


start = time.time()
threads = []

for url in urls:
    thread = threading.Thread(target=read_url, args=(url,))
    thread.start()
    threads.append(thread)
for thread in threads:
    thread.join()

print(time.time() - start)

3.89656138420105


время изначального кода: 5.62566065788269

время моего варианта с потоками: 3.89656138420105

**Упражнение 5.**

Составить программу, которая имеет общие ресурсы для нескольких потоков. Например, есть общая переменная, один поток добавляет 1, второй увеличивает значение в 2 раза. Написать с использованием Lock. Продемонстрировать проблему взаимной блокировки. Исправить её, написав код с использованием RLock блокировки.

In [ ]:
import threading

def add_one():
  global resource
  for _ in range(10):
    with lock:
      resource += 1
      print(f"resource +1 = {resource}")

def add_two():
  global resource
  for _ in range(10):
    with lock:
      resource += 2
      print(f"resource +2 = {resource}")


def start_calc():
  with lock:
    add_one()
    add_two()

resource = 0 #заданный ресурс/переменная
lock = threading.Lock()

thread1 = threading.Thread(target=start_calc)
thread2 = threading.Thread(target=start_calc)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print(f"Final: {resource}")

Lock не знает, какой поток блокируется в данный момент и долго грузит/стоит

In [ ]:
import threading

def add_one():
  global resource
  for _ in range(10):
    with rlock:
      resource += 1
      print(f"resource +1 = ", resource)

def add_two():
  global resource
  for _ in range(10):
    with rlock:
      resource += 2
      print(f"resource +2 = ", resource)


def start_calc():
  with rlock:
    add_one()
    add_two()

resource = 0 #заданный ресурс/переменная
rlock = threading.RLock() # lock поменяли на rlock
thread1 = threading.Thread(target=start_calc)
thread2 = threading.Thread(target=start_calc)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print(f"Final: {resource}")

resource +1 =  1
resource +1 =  2
resource +1 =  3
resource +1 =  4
resource +1 =  5
resource +1 =  6
resource +1 =  7
resource +1 =  8
resource +1 =  9
resource +1 =  10
resource +2 =  12
resource +2 =  14
resource +2 =  16
resource +2 =  18
resource +2 =  20
resource +2 =  22
resource +2 =  24
resource +2 =  26
resource +2 =  28
resource +2 =  30
resource +1 =  31
resource +1 =  32
resource +1 =  33
resource +1 =  34
resource +1 =  35
resource +1 =  36
resource +1 =  37
resource +1 =  38
resource +1 =  39
resource +1 =  40
resource +2 =  42
resource +2 =  44
resource +2 =  46
resource +2 =  48
resource +2 =  50
resource +2 =  52
resource +2 =  54
resource +2 =  56
resource +2 =  58
resource +2 =  60
Final: 60
